In [1]:
import os
import sys
import torch
import importlib

from dotenv import load_dotenv
from datasets import Dataset

In [2]:
# Setup module path for local imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Load environment variables
load_dotenv()

# Configure device
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

# Fix random seed for reproducibility
torch.manual_seed(0)

In [3]:
device

'mps'

In [4]:
import src.train
import src.model
import data.zebra
import evals.zebra_eval

importlib.reload(src.train)
importlib.reload(src.model)
importlib.reload(data.zebra)
importlib.reload(evals.zebra_eval)

from src.train import sft_train_lora
from src.model import identify_target_modules
from data.zebra import Zebra
from evals.zebra_eval import compute_zebra_metrics, eval_model_zebra, ZebraPuzzleMetric
from data.format import chat_format_qa_instance, lm_format_qa_instance

In [5]:
def load_prep_zebra_dataset(tokenizer, instruction_tuned=True, test_split_size=0.2):
    dataset = Zebra(hf_token=os.environ["HF_TOKEN"])
    if instruction_tuned:
        formatted_list = [chat_format_qa_instance(example) for example in dataset]
        formatted_list = tokenizer.apply_chat_template(
            formatted_list, tokenize=False, add_generation_prompt=False
        )
    else:
        formatted_list = [lm_format_qa_instance(example) for example in dataset]
    dataset = Dataset.from_dict({"formatted_text": formatted_list})

    dataset = dataset.train_test_split(test_size=test_split_size)
    return dataset

In [6]:
correct_response = """The solution is as follows:
In house 1, name is Eric, pet is bird.
In house 2, name is Arnold, pet is dog.
In house 3, name is Alice, pet is cat.
In house 4, name is Peter, pet is fish."""

In [7]:
varied_correct_responses = [
    correct_response,
    """	The solution is structured as follows:
	•	House 1: Eric is the resident, and his pet is a bird.
	•	House 2: Arnold lives here, and he has a dog as a pet.
	•	House 3: Alice occupies this house, and her pet is a cat.
	•	House 4: Peter resides in this house, and his pet is a fish.""",
    """ Here is the breakdown of the solution:
	•	In House 1, Eric lives with his pet bird.
	•	House 2 is home to Arnold, who has a dog.
	•	House 3 is where Alice resides, along with her cat.
	•	Peter lives in House 4, and his pet is a fish.""",
    """ The solution can be presented as follows:
	•	House 1 belongs to Eric, and he keeps a bird as a pet.
	•	Arnold stays in House 2, with a dog as his pet.
	•	House 3 is where Alice lives, and she has a cat.
	•	In House 4, Peter resides, and he owns a fish."""
]
references = [correct_response, correct_response, correct_response, correct_response]
assert len(varied_correct_responses) == len(references)

In [8]:
# Compute the metrics
metrics = compute_zebra_metrics(varied_correct_responses, references)
metrics

part pairs: {1: {'ref': 'In house 1, name is Eric, pet is bird', 'pred': 'In house 1, name is Eric, pet is bird'}, 2: {'ref': 'In house 2, name is Arnold, pet is dog', 'pred': 'In house 2, name is Arnold, pet is dog'}, 3: {'ref': 'In house 3, name is Alice, pet is cat', 'pred': 'In house 3, name is Alice, pet is cat'}, 4: {'ref': 'In house 4, name is Peter, pet is fish', 'pred': 'In house 4, name is Peter, pet is fish'}}
part pairs: {1: {'ref': 'In house 1, name is Eric, pet is bird', 'pred': 'House 1: Eric is the resident, and his pet is a bird'}, 2: {'ref': 'In house 2, name is Arnold, pet is dog', 'pred': 'House 2: Arnold lives here, and he has a dog as a pet'}, 3: {'ref': 'In house 3, name is Alice, pet is cat', 'pred': 'House 3: Alice occupies this house, and her pet is a cat'}, 4: {'ref': 'In house 4, name is Peter, pet is fish', 'pred': 'House 4: Peter resides in this house, and his pet is a fish'}}
part pairs: {1: {'ref': 'In house 1, name is Eric, pet is bird', 'pred': 'In Hou

{'eval_strict_accuracy': 1.0, 'eval_partial_accuracy': 1.0}

In [9]:
varied_incorrect_responses = [
    correct_response,
    """	The solution is structured as follows:
	•	House 1: Bob is the resident, and his pet is a bird. 
	•	House 2: Arnold lives here, and he has a cat as a pet.
	•	House 3: Alice occupies this house, and her pet is a cat.
	•	House 4: Peter resides in this house, and his pet is a fish.""",
    """ Here is the breakdown of the solution:
	•	In House 1, Janson lives with his pet bird.
	•	House 2 is home to Arnold, who has a dog.
	•	House 3 is where Alice resides, along with her cat.
	•	Peter lives in House 4, and his pet is a fish.""",
    """ The solution can be presented as follows:
	•	House 1 belongs to Eric, and he keeps a bird as a pet.
	•	Arnold stays in House 2, with a dog as his pet.
	•	House 3 is where Alice lives, and she has a cat.
	•	In House 4, Peter resides, and he owns a fish."""
]

In [10]:
# Compute the metrics
metrics = compute_zebra_metrics(varied_incorrect_responses, references)
metrics

part pairs: {1: {'ref': 'In house 1, name is Eric, pet is bird', 'pred': 'In house 1, name is Eric, pet is bird'}, 2: {'ref': 'In house 2, name is Arnold, pet is dog', 'pred': 'In house 2, name is Arnold, pet is dog'}, 3: {'ref': 'In house 3, name is Alice, pet is cat', 'pred': 'In house 3, name is Alice, pet is cat'}, 4: {'ref': 'In house 4, name is Peter, pet is fish', 'pred': 'In house 4, name is Peter, pet is fish'}}
part pairs: {1: {'ref': 'In house 1, name is Eric, pet is bird', 'pred': 'House 1: Bob is the resident, and his pet is a bird'}, 2: {'ref': 'In house 2, name is Arnold, pet is dog', 'pred': 'House 2: Arnold lives here, and he has a cat as a pet'}, 3: {'ref': 'In house 3, name is Alice, pet is cat', 'pred': 'House 3: Alice occupies this house, and her pet is a cat'}, 4: {'ref': 'In house 4, name is Peter, pet is fish', 'pred': 'House 4: Peter resides in this house, and his pet is a fish'}}
part pairs: {1: {'ref': 'In house 1, name is Eric, pet is bird', 'pred': 'In Hous

{'eval_strict_accuracy': 0.5, 'eval_partial_accuracy': 0.90625}